## Dictionary Leanring for images
Input: Image

Output: Dictionary atoms, sparse matrix 

Prepare image dataset for Dictionray Leanring

For images, we just need an addtional preprocessing

## Utils
1.Preprocess image to a 2D merged large matrix.

## Note: we only provide preprocess function, the remaining process is similar to the TEM signal example.

In [3]:
import os
import torch
import numpy as np
from PIL import Image
from tqdm import tqdm

def load_images_from_folder(image_path:str=None, crop_size:tuple=(512, 512), block_size:int=8):
    """
    image_path (str): Image file path, default: None.
    crop_size (tuple): Cropping size setting of the image, default: (512,512).
    block_size (int): Image block size setting, default: 8.
    """
    assert os.path.isdir(image_path), f"Error: {image_path}, Images loading failed."
    
    """Loading images, e,g. 00001.jpg"""
    images = []
    for filename in os.listdir(image_path):
        if filename.lower().endswith(".jpg"):  #search files matching for .jpg files
            file_path = os.path.join(image_path, filename)
            try:
                image = Image.open(file_path).convert("L").resize(crop_size)
                image = np.array(image) / 255.0 
                images.append(image)
            except Exception as e:
                print(f"Error loading image {filename}: {e}")
                continue
    assert len(images) > 0, "Error: No valid images were loaded."
    
    """Merge images"""
    images = np.stack(images, axis=0)

    """
    Divide the image into small block, e.g., (512,512) -> (64,64) x (8,8) blocks -> flatten -> (4096,64) reshape feature matrix
    Note: In provided Image_dataset, there are two images, so the final matrix should be: ((4096x2),64)->(8192,64)

    """
    blocks = []
    for img in images:
        for i in range(0, crop_size[0], block_size):
            for j in range(0, crop_size[1], block_size):
                block = img[i:i + block_size, j:j + block_size]
                blocks.append(block.flatten()) #
    blocks = np.stack(blocks)
    print(f"The transfored image matrix shape: {blocks.shape}")
    return torch.tensor(blocks, dtype=torch.float32)

#Example
image_folder_path = "Image_dataset"  #Replace with your own path
blocks_tensor = load_images_from_folder(image_folder_path)

The transfored image matrix shape: (8192, 64)
